# Panel Data

## Pre-Processing

In [24]:
# Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import datetime
import math

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

#from linearmodels.panel import PanelOLS, RandomEffects
from tabulate import tabulate

import warnings
warnings.filterwarnings('ignore')

In [25]:
def calculate_aic(y_true, y_pred, num_features):
    resid = y_true - y_pred
    sse = np.sum(resid ** 2)
    aic = len(y_true) * np.log(sse / len(y_true)) + 2 * num_features
    return aic


def calculate_bic(y_true, y_pred, num_features):
    resid = y_true - y_pred
    sse = np.sum(resid ** 2)
    n = len(y_true)
    bic = n * np.log(sse / n) + num_features * np.log(n)
    return bic


def forward_step(X_train, y_train, selected_features, remaining_features, criterion_func):
    best_criterion = np.inf
    best_feature = None
    
    for feature in remaining_features:
        temp_features = selected_features + [feature]
        temp_model = sm.OLS(y_train, sm.add_constant(X_train[temp_features])).fit()
        temp_criterion = criterion_func(y_train, temp_model.predict(sm.add_constant(X_train[temp_features])), len(temp_model.params))
        
        if temp_criterion < best_criterion:
            best_criterion = temp_criterion
            best_feature = feature
    
    return best_criterion, best_feature

def backward_step(X_train, y_train, selected_features, criterion_func):
    best_criterion = np.inf
    best_feature = None
    
    for feature in selected_features:
        temp_features = selected_features.copy()
        temp_features.remove(feature)
        temp_model = sm.OLS(y_train, sm.add_constant(X_train[temp_features])).fit()
        temp_criterion = criterion_func(y_train, temp_model.predict(sm.add_constant(X_train[temp_features])), len(temp_model.params))
        
        if temp_criterion < best_criterion:
            best_criterion = temp_criterion
            best_feature = feature
    
    return best_criterion, best_feature

def stepwise_bidirectional_selection(X_train, y_train, method='aic'):
    features = list(X_train.columns)
    selected_features = []
    best_criterion = np.inf
    
    while len(features) > 0:
        if method == 'aic':
            forward_criterion, forward_feature = forward_step(X_train, y_train, selected_features, features, calculate_aic)
            backward_criterion, backward_feature = backward_step(X_train, y_train, selected_features, calculate_aic)
        elif method == 'bic':
            forward_criterion, forward_feature = forward_step(X_train, y_train, selected_features, features, calculate_bic)
            backward_criterion, backward_feature = backward_step(X_train, y_train, selected_features, calculate_bic)
        else:
            raise ValueError("Invalid criterion_func. Use 'aic' or 'bic'.")
        
        if forward_criterion < backward_criterion and forward_criterion < best_criterion:
            selected_features.append(forward_feature)
            best_criterion = forward_criterion
        elif backward_criterion < forward_criterion and backward_criterion < best_criterion:
            selected_features.remove(backward_feature)
            best_criterion = backward_criterion
        else:
            break
    
    return selected_features


def mape(y_true, y_pred):
    non_zero_indices = np.where(y_true != 0)[0]
    y_true_no_zeros = np.array(y_true)[non_zero_indices]
    y_pred_no_zeros = np.array(y_pred)[non_zero_indices]

    absolute_percentage_errors = np.abs((y_true_no_zeros - y_pred_no_zeros) / y_true_no_zeros)
    mape_value = np.mean(absolute_percentage_errors) * 100
    return mape_value

def mae(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [26]:
df = pd.read_csv('data_sensors_rovere.csv')
df = df.rename(columns={'group': 'group_id'})

df_rovere = df[['reading_id', 'timestamp', 'sensor_id', 'value', 'description', 'group_id']]

df_rovere['reading_id'] = df_rovere['reading_id'].astype(str)
df_rovere['timestamp'] = pd.to_datetime(df_rovere['timestamp']).dt.floor('D').dt.date
df_rovere['sensor_id'] = df_rovere['sensor_id'].astype(str)
df_rovere['value'] = df_rovere['value'].astype(float)
df_rovere['description'] = df_rovere['description'].astype(str)
df_rovere['group_id'] = df_rovere['group_id'].astype(str)

condition_30 = df_rovere['sensor_id'].isin(['72', '76', '73', '74', '61', '63', '67', '65'])
condition_60 = df_rovere['sensor_id'].isin(['71', '69', '75', '70', '62', '64', '68', '66'])
condition_irrigation = df_rovere['description'] == 'irrigation'

df_rovere.loc[condition_30, 'description'] = 'Tensiometer 30'
df_rovere.loc[condition_60, 'description'] = 'Tensiometer 60'
df_rovere.loc[condition_irrigation, 'description'] = 'Irrigation'

print('Shape:', df_rovere.shape)
print('Types:\n', df_rovere.dtypes)
df_rovere.head(10)

In [27]:
condition_not_in_list = ~df_rovere['sensor_id'].isin(['72', '76', '73', '74', '61', '63', '67', '65', '71', '69', '75', '70', '62', '64', '68', '66'])
df_to_duplicate = df_rovere[condition_not_in_list]
df_to_duplicate['group_id_1'] = df_to_duplicate['group_id'] + '_1'

df_rovere = pd.concat([df_rovere, df_to_duplicate], ignore_index=True)
df_rovere.sort_values(by=['group_id', 'timestamp'], inplace=True)
df_rovere.reset_index(drop=True, inplace=True)

condition_group_id_1 = df_rovere['group_id_1'].notnull()
df_rovere.loc[condition_group_id_1, 'group_id'] = df_rovere.loc[condition_group_id_1, 'group_id_1']
df_rovere.drop(columns=['group_id_1'], inplace=True)

condition_update_group_id = df_rovere['sensor_id'].isin(['71', '69', '75', '70', '62', '64', '68', '66'])
df_rovere.loc[condition_update_group_id, 'group_id'] = df_rovere.loc[condition_update_group_id, 'group_id'] + '_1'


df_group = df_rovere.groupby(['timestamp', 'description', 'sensor_id', 'group_id']).agg({'value': ['min', 'max', 'mean', 'median', 'sum']}).reset_index()
df_group.columns = ['timestamp', 'description', 'sensor_id', 'group_id', 'val_min', 'val_max', 'val_avg', 'val_med', 'val_sum']

df_pivot = df_group.pivot(index=['timestamp', 'group_id'], columns='description', values=['val_min', 'val_max', 'val_avg', 'val_med', 'val_sum']).reset_index()
df_pivot.columns.name = None
df_pivot.columns = ['date', 'group_id', 'min_hum', 'min_temp', 'min_solar', 'min_irr', 'min_rain', 'min_tens30', 'min_tens60',
                    'max_hum', 'max_temp', 'max_solar', 'max_irr', 'max_rain', 'max_tens30', 'max_tens60',
                    'avg_hum', 'avg_temp', 'avg_solar', 'avg_irr', 'avg_rain', 'avg_tens30', 'avg_tens60',
                    'med_hum', 'med_temp', 'med_solar', 'med_irr', 'med_rain', 'med_tens30', 'med_tens60',
                    'sum_hum', 'sum_temp', 'sum_solar', 'sum_irr', 'sum_rain', 'sum_tens30', 'sum_tens60']

df_pivot['min_tens'] = df_pivot['min_tens30'].combine_first(df_pivot['min_tens60'])
df_pivot.drop(columns=['min_tens30', 'min_tens60'], inplace=True)


df_pivot['max_tens'] = df_pivot['max_tens30'].combine_first(df_pivot['max_tens60'])
df_pivot.drop(columns=['max_tens30', 'max_tens60'], inplace=True)


df_pivot['avg_tens'] = df_pivot['avg_tens30'].combine_first(df_pivot['avg_tens60'])
df_pivot.drop(columns=['avg_tens30', 'avg_tens60'], inplace=True)


df_pivot['med_tens'] = df_pivot['med_tens30'].combine_first(df_pivot['med_tens60'])
df_pivot.drop(columns=['med_tens30', 'med_tens60'], inplace=True)


df_pivot['sum_tens'] = df_pivot['sum_tens30'].combine_first(df_pivot['sum_tens60'])
df_pivot.drop(columns=['sum_tens30', 'sum_tens60'], inplace=True)

df_pivot = df_pivot.dropna().reset_index(drop=True)

df = df_pivot

columns_to_drop = ['min_irr', 'max_irr', 'avg_irr', 'med_irr', 'min_rain', 'sum_hum', 'sum_temp', 'sum_solar', 'avg_rain']
df = df.drop(columns=columns_to_drop).dropna().reset_index(drop=True)


group_id_mapping = {
    '1': '72',
    '2': '76',
    '3': '73',
    '4': '74',
    '5': '61',
    '6': '63',
    '7': '67',
    '8': '65',
    '1_1': '71',
    '2_1': '69',
    '3_1': '75',
    '4_1': '70',
    '5_1': '62',
    '6_1': '64',
    '7_1': '68',
    '8_1': '66'
}

df['group_id'] = df['group_id'].replace(group_id_mapping)
df = df.rename(columns={'group_id': 'sensor_id'})
df = df[['sensor_id', 'date', 'avg_tens'] + [col for col in df.columns if col not in ['sensor_id', 'date', 'avg_tens']]]
df = df.sort_values(by=['sensor_id', 'date']).reset_index(drop=True)

df

## Regression

In [28]:
ids = df['sensor_id']
dates = df['date']
X = df.drop(columns=['date', 'sensor_id'])
X = X.shift(1).add_suffix('_lag1').join(X.shift(2).add_suffix('_lag2')).join(X.shift(3).add_suffix('_lag3'))
X['id'] = ids
X['date'] = dates

dates_to_remove = [datetime.date(2023, 4, 28), datetime.date(2023, 4, 29), datetime.date(2023, 4, 30)]
X = X[~X['date'].isin(dates_to_remove)]
X = X.sort_values(by=['date', 'id'])
X = X.drop(columns=['id', 'date']).dropna().reset_index(drop=True)

y = df[['avg_tens', 'date', 'sensor_id']]
y = y[~y['date'].isin(dates_to_remove)]
y = y.sort_values(by=['date', 'sensor_id'])
y = y.drop(columns=['sensor_id', 'date']).dropna().reset_index(drop=True)
y = y.squeeze()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.29878, shuffle=False)

In [29]:
all_features_model = sm.OLS(y_train, sm.add_constant(X_train)).fit()
all_features_predictions = all_features_model.predict(sm.add_constant(X_train))
all_features_predictions_test = all_features_model.predict(sm.add_constant(X_test))


selected_features = stepwise_bidirectional_selection(X_train, y_train, method='aic')
final_model = sm.OLS(y_train, sm.add_constant(X_train[selected_features])).fit()

train_linear_pred = final_model.predict(sm.add_constant(X_train[selected_features]))
linear_pred = final_model.predict(sm.add_constant(X_test[selected_features]))


print(f'Number of Selected Features: {len(selected_features)}')
print("Selected Features:")
for feature in selected_features:
    print(f" - {feature}")

In [30]:
ids = df['sensor_id']
dates = df['date']
X = df.drop(columns=['date', 'sensor_id'])
X = X.shift(1).add_suffix('_lag1').join(X.shift(2).add_suffix('_lag2')).join(X.shift(3).add_suffix('_lag3'))
X['id'] = ids
X['date'] = dates

dummies = pd.get_dummies(X['id'], drop_first=True).astype(int)
dummy_features = dummies.columns.tolist()
X = X.join(dummies)

dates_to_remove = [datetime.date(2023, 4, 28), datetime.date(2023, 4, 29), datetime.date(2023, 4, 30)]
X = X[~X['date'].isin(dates_to_remove)].dropna().reset_index(drop=True)
X = X.sort_values(by=['date', 'id']).dropna().reset_index(drop=True)
X = X.drop(columns=['id', 'date'])

y = df[['avg_tens', 'date', 'sensor_id']]
y = y[~y['date'].isin(dates_to_remove)]
y = y.sort_values(by=['date', 'sensor_id'])
y = y.drop(columns=['sensor_id', 'date']).dropna().reset_index(drop=True)
y = y.squeeze()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.29878, shuffle=False)

In [31]:
selected_features = stepwise_bidirectional_selection(X_train, y_train, method='aic')

model = sm.OLS(y_train, X_train[selected_features]).fit()

train_pool_pred = model.predict(X_train[selected_features])
pool_pred = model.predict(X_test[selected_features])


print(f'Number of Selected Features: {len(selected_features)}')
print("Selected Features:")
for feature in selected_features:
    print(f" - {feature}")

In [32]:
print(model.summary())

In [33]:
ids = df['sensor_id']
dates = df['date']
X = df.drop(columns=['date', 'sensor_id'])
X = X.shift(1).add_suffix('_lag1').join(X.shift(2).add_suffix('_lag2')).join(X.shift(3).add_suffix('_lag3'))
X['id'] = ids
X['date'] = dates

dummies = pd.get_dummies(X['id'], drop_first=True).astype(int)
dummy_features = dummies.columns.tolist()
X = X.join(dummies)

dates_to_remove = [datetime.date(2023, 4, 28), datetime.date(2023, 4, 29), datetime.date(2023, 4, 30)]
X = X[~X['date'].isin(dates_to_remove)].dropna().reset_index(drop=True)
X = X.sort_values(by=['date', 'id']).dropna().reset_index(drop=True)
ids_sorted = X['id']
dates_sorted = X['date']
X = X.drop(columns=['id', 'date'])

y = df[['avg_tens', 'date', 'sensor_id']]
y = y[~y['date'].isin(dates_to_remove)]
y = y.sort_values(by=['date', 'sensor_id'])
y = y.drop(columns=['sensor_id', 'date']).dropna().reset_index(drop=True)
y = y.squeeze()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.29878, shuffle=False)

In [34]:
X_train_no_dummy = X_train.drop(dummy_features, axis=1)

In [35]:
X_train_ids = pd.concat([ids_sorted[:1840], X_train_no_dummy], axis=1)
y_train_ids = pd.concat([ids_sorted[:1840], y_train], axis=1)

In [36]:
y_train_ids

In [37]:
X_train.shape

In [38]:
sensor_names = ['72', '76', '73', '74', '61', '63', '67', '65', '71', '69', '75', '70', '62', '64', '68', '66']
n = len(sensor_names)
T = X_train_no_dummy.shape[0] / n
N=n*T
k = X_train_no_dummy.shape[1] + 1
k

In [39]:
X_train

In [40]:
# Step 1

lm_model = sm.OLS(y_train, X_train_no_dummy).fit()
lm_residuals = lm_model.resid

sigma2_lm = lm_model.ssr/(n*T-(k+1))
print('sigma2_pooled = ' + str(sigma2_lm))

In [41]:
# Step 2

fe_model = sm.OLS(y_train, X_train).fit()

sigma2_epsilon = fe_model.ssr/(n*T-(n+k+1))
print('sigma2_epsilon = ' + str(sigma2_epsilon))

sigma2_u = sigma2_lm - sigma2_epsilon
print('sigma2_u = ' + str(sigma2_u))

In [42]:
# Step 3

X_group_means = X_train_ids.groupby('id').mean()
X_group_means

In [43]:
y_group_means = y_train_ids.groupby('id').mean()
y_group_means

In [44]:
# Step 4

theta = 1 - math.sqrt(sigma2_epsilon/(sigma2_epsilon + T*sigma2_u))
print('theta = ' + str(theta))

In [45]:
# sigma2_u / (sigma2_u + sigma2_epsilon)

In [46]:
mape_train_linear_all = round(mape(y_train, all_features_predictions), 3)
mae_train_linear_all = round(mae(y_train, all_features_predictions), 3)
rmse_train_linear_all = round(rmse(y_train, all_features_predictions), 3)

mape_train_linear = round(mape(y_train, train_linear_pred), 3)
mae_train_linear = round(mae(y_train, train_linear_pred), 3)
rmse_train_linear = round(rmse(y_train, train_linear_pred), 3)

mape_train_pool = round(mape(y_train, train_pool_pred), 3)
mae_train_pool = round(mae(y_train, train_pool_pred), 3)
rmse_train_pool = round(rmse(y_train, train_pool_pred), 3)


mape_linear_all = round(mape(y_test, all_features_predictions_test), 3)
mae_linear_all = round(mae(y_test, all_features_predictions_test), 3)
rmse_linear_all = round(rmse(y_test, all_features_predictions_test), 3)

mape_linear = round(mape(y_test, linear_pred), 3)
mae_linear = round(mae(y_test, linear_pred), 3)
rmse_linear = round(rmse(y_test, linear_pred), 3)

mape_pool = round(mape(y_test, pool_pred), 3)
mae_pool = round(mae(y_test, pool_pred), 3)
rmse_pool = round(rmse(y_test, pool_pred), 3)


table = [
    ['Metric', 'LM Train ALL', 'LM Train', 'Pool Train', 'LM Test ALL', 'LM Test', 'Pool Test'],
    ['MAPE', mape_train_linear_all, mape_train_linear, mape_train_pool, mape_linear_all, mape_linear, mape_pool],
    ['MAE', mae_train_linear_all, mae_train_linear, mae_train_pool, mae_linear_all, mae_linear, mae_pool],
    ['RMSE', rmse_train_linear_all, rmse_train_linear, rmse_train_pool, rmse_linear_all, rmse_linear, rmse_pool]
]

print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))